In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

Our dataset will be a simple $R^d$ Gaussian distribution

Formula = (W - F + 2P) / S + 1

W - input volume size

F - the receptive field size of convnet neurons 

S - stride

P - zero padding 

## 1D teacher student

In [168]:
class MyConv(nn.Module):
    def __init__(self, d=100):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=2)
        self.fc = nn.Linear(102,1)
        self.sigmoid=nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

In [169]:
from copy import deepcopy
class MyLin(nn.Module):
    def __init__(self, d=100):
        super().__init__()
        self.lin1 = conv_to_fc(myconv.conv1,(1,100))
        self.fc = deepcopy(myconv.fc)
        self.sigmoid=nn.Sigmoid()

    def forward(self, x):
        x = self.lin1(x)
        x = F.relu(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

In [170]:
class BasicLin(nn.Module):
    def __init__(self, d=100):
        super().__init__()
        self.lin1 = nn.Linear(100,102)
        self.fc = nn.Linear(102,1)
        self.sigmoid=nn.Sigmoid()

    def forward(self, x):
        x = self.lin1(x)
        x = F.relu(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

input dimension

In [213]:
d=100

In [219]:
train_inp = torch.randn((10000,1,d))
test_inp = torch.randn((1000,1,d))

In [220]:
myconv = MyConv()
lin = BasicLin()

In [221]:
mylin=MyLin(100)

In [226]:
train_conv_out = myconv(train_inp).squeeze()

In [229]:
conv_out = myconv(test_inp).squeeze()
lin_out = mylin(test_inp.view(test_inp.shape[0], -1)).squeeze()
torch.norm( conv_out - lin_out)

tensor(6.9893e-07, grad_fn=<LinalgVectorNormBackward0>)

## This shows that the FC CAN learn the conv structure

Error before training is

In [230]:
fcn_out = lin(test_inp).squeeze()
torch.norm( conv_out - fcn_out)

tensor(3.1877, grad_fn=<LinalgVectorNormBackward0>)

In [231]:
conv_out - fcn_out

tensor([-0.0707,  0.0858, -0.1088,  0.0968, -0.0704,  0.0216, -0.1047,  0.0226,
         0.0309,  0.2256, -0.0727,  0.0913, -0.0106, -0.0825,  0.1376,  0.0880,
         0.0489,  0.1471,  0.1502, -0.0561, -0.0201, -0.0057, -0.1607,  0.0069,
         0.0849,  0.0797, -0.1893,  0.0718,  0.0922, -0.0095, -0.1465,  0.0441,
         0.0563,  0.0671,  0.0305, -0.0081, -0.0999, -0.1324,  0.0038, -0.0586,
        -0.0948, -0.0081,  0.1312, -0.0924,  0.0121,  0.1259, -0.1047,  0.0248,
         0.1011,  0.1123,  0.0964,  0.0390,  0.0312,  0.1611,  0.2392, -0.1767,
         0.0049,  0.0680, -0.0114, -0.0797, -0.0288,  0.0034,  0.0549,  0.0969,
         0.0498, -0.0565,  0.1016, -0.0531,  0.0886,  0.0311, -0.0299, -0.2000,
        -0.0055,  0.0434, -0.0455,  0.0405,  0.1540, -0.0656, -0.0684, -0.0226,
         0.0409,  0.1832,  0.0131, -0.0100,  0.1175,  0.0500, -0.2297,  0.0216,
        -0.0512,  0.0606, -0.0586,  0.1383, -0.0055, -0.0414, -0.1222,  0.0556,
         0.1194, -0.0292,  0.0585,  0.01

## Train FCN

In [148]:
mse = torch.nn.MSELoss()
#optimizer = torch.optim.SGD(lin.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(lin.parameters(), lr=0.001)
num_epochs = 10000

In [149]:
for epoch in range(num_epochs):
    
    # take random 128 batch
    batch_size=10000
    inputs = train_inp[torch.randint(0, train_inp.shape[0], (batch_size,))]
    labels = myconv(inputs).unsqueeze(-1)
    out = lin(inputs)
    loss = mse(out, labels)
    
    ## Backwards pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if num_epochs%100==0: 
        
        fcn_out = lin(train_inp).squeeze()
        test_mse = mse( train_conv_out - fcn_out)
        print(f'Train MSE is {test_mse}\n')
        
        fcn_out = lin(test_inp).squeeze()
        test_mse = mse(conv_out - fcn_out)
        print(f'Test MSE is {test_mse}\n\n')
print('Finished Training')

Train MSE is 7.540276050567627

Test MSE is 2.412055015563965


Train MSE is 6.81891393661499

Test MSE is 2.182281970977783


Train MSE is 6.17057466506958

Test MSE is 1.9754294157028198


Train MSE is 5.612314701080322

Test MSE is 1.796743392944336


Train MSE is 5.157586574554443

Test MSE is 1.6503251791000366


Train MSE is 4.8097100257873535

Test MSE is 1.5375937223434448


Train MSE is 4.558272361755371

Test MSE is 1.4553838968276978


Train MSE is 4.38143253326416

Test MSE is 1.396960973739624


Train MSE is 4.25165319442749

Test MSE is 1.3538316488265991


Train MSE is 4.14469575881958

Test MSE is 1.3181517124176025


Train MSE is 4.042015552520752

Test MSE is 1.2839857339859009


Train MSE is 3.937376022338867

Test MSE is 1.249368667602539


Train MSE is 3.829261302947998

Test MSE is 1.213731050491333


Train MSE is 3.721539258956909

Test MSE is 1.1781275272369385


Train MSE is 3.620534896850586

Test MSE is 1.144787073135376


Train MSE is 3.5327274799346924

Tes

Test MSE is 0.7424283027648926


Train MSE is 1.7846201658248901

Test MSE is 0.7414121031761169


Train MSE is 1.7779467105865479

Test MSE is 0.7404226660728455


Train MSE is 1.7713583707809448

Test MSE is 0.7394833564758301


Train MSE is 1.764671802520752

Test MSE is 0.7384357452392578


Train MSE is 1.7579929828643799

Test MSE is 0.7373488545417786


Train MSE is 1.7512717247009277

Test MSE is 0.7364339828491211


Train MSE is 1.7444989681243896

Test MSE is 0.7356274127960205


Train MSE is 1.737784504890442

Test MSE is 0.7346203327178955


Train MSE is 1.7310218811035156

Test MSE is 0.7336505651473999


Train MSE is 1.7244001626968384

Test MSE is 0.7326852083206177


Train MSE is 1.717847228050232

Test MSE is 0.731867790222168


Train MSE is 1.711392879486084

Test MSE is 0.7310436367988586


Train MSE is 1.7050580978393555

Test MSE is 0.7301402688026428


Train MSE is 1.6987476348876953

Test MSE is 0.7292120456695557


Train MSE is 1.6924687623977661

Test MSE is 0.7

Test MSE is 0.6655356884002686


Train MSE is 1.1825754642486572

Test MSE is 0.6651334166526794


Train MSE is 1.1794825792312622

Test MSE is 0.6646571159362793


Train MSE is 1.1765496730804443

Test MSE is 0.6643224954605103


Train MSE is 1.1736211776733398

Test MSE is 0.6640019416809082


Train MSE is 1.1706788539886475

Test MSE is 0.6634911894798279


Train MSE is 1.1675573587417603

Test MSE is 0.6628530621528625


Train MSE is 1.1644514799118042

Test MSE is 0.6623160243034363


Train MSE is 1.1614962816238403

Test MSE is 0.6617234945297241


Train MSE is 1.1586427688598633

Test MSE is 0.6612402200698853


Train MSE is 1.1557238101959229

Test MSE is 0.6607327461242676


Train MSE is 1.1529639959335327

Test MSE is 0.660349428653717


Train MSE is 1.150083303451538

Test MSE is 0.6598215103149414


Train MSE is 1.1469637155532837

Test MSE is 0.6593534350395203


Train MSE is 1.144067645072937

Test MSE is 0.6589617133140564


Train MSE is 1.1412991285324097

Test MSE is 0

Test MSE is 0.6307281255722046


Train MSE is 0.8964744210243225

Test MSE is 0.6308498382568359


Train MSE is 0.8947542905807495

Test MSE is 0.6308538913726807


Train MSE is 0.8930788040161133

Test MSE is 0.6307751536369324


Train MSE is 0.8914710879325867

Test MSE is 0.63046795129776


Train MSE is 0.8900986909866333

Test MSE is 0.6300597190856934


Train MSE is 0.889164924621582

Test MSE is 0.6294785737991333


Train MSE is 0.8880944848060608

Test MSE is 0.6287063956260681


Train MSE is 0.8865423202514648

Test MSE is 0.6279318332672119


Train MSE is 0.8846237659454346

Test MSE is 0.6271706819534302


Train MSE is 0.8825365304946899

Test MSE is 0.626609206199646


Train MSE is 0.8804692029953003

Test MSE is 0.626154363155365


Train MSE is 0.8786946535110474

Test MSE is 0.626008927822113


Train MSE is 0.8773889541625977

Test MSE is 0.6259952187538147


Train MSE is 0.8763304352760315

Test MSE is 0.6261720657348633


Train MSE is 0.8752665519714355

Test MSE is 0.62

Test MSE is 0.6131710410118103


Train MSE is 0.7389611005783081

Test MSE is 0.6130560636520386


Train MSE is 0.7377069592475891

Test MSE is 0.61322021484375


Train MSE is 0.736464262008667

Test MSE is 0.6135545372962952


Train MSE is 0.7354352474212646

Test MSE is 0.6138959527015686


Train MSE is 0.7343865036964417

Test MSE is 0.6140092611312866


Train MSE is 0.7334447503089905

Test MSE is 0.6140909194946289


Train MSE is 0.7326891422271729

Test MSE is 0.6142687797546387


Train MSE is 0.7319633364677429

Test MSE is 0.6144108772277832


Train MSE is 0.731060802936554

Test MSE is 0.6143685579299927


Train MSE is 0.7301014065742493

Test MSE is 0.6142957210540771


Train MSE is 0.7290319204330444

Test MSE is 0.6139724254608154


Train MSE is 0.7279390096664429

Test MSE is 0.6134522557258606


Train MSE is 0.7266802787780762

Test MSE is 0.6128994822502136


Train MSE is 0.7255072593688965

Test MSE is 0.6124591827392578


Train MSE is 0.7246601581573486

Test MSE is 0.

...

Train MSE is 0.24379289150238037

Test MSE is 0.6409667134284973


Train MSE is 0.24378256499767303

Test MSE is 0.6402936577796936


Train MSE is 0.24330981075763702

Test MSE is 0.6397684812545776


Train MSE is 0.2434675395488739

Test MSE is 0.639877438545227


Train MSE is 0.2444581389427185

Test MSE is 0.6404431462287903


Train MSE is 0.24323278665542603

Test MSE is 0.6410988569259644


Train MSE is 0.24225683510303497

Test MSE is 0.641390860080719


Train MSE is 0.2431560903787613

Test MSE is 0.6414995193481445


Train MSE is 0.24241605401039124

Test MSE is 0.6413117051124573


Train MSE is 0.24298825860023499

Test MSE is 0.6410083174705505


Train MSE is 0.24299505352973938

Test MSE is 0.6408585906028748


Train MSE is 0.24255383014678955

Test MSE is 0.6407767534255981


Train MSE is 0.24166537821292877

Test MSE is 0.6408513784408569


Train MSE is 0.24325266480445862

Test MSE is 0.6409997344017029


Train MSE is 0.24281823635101318

Test MSE is 0.6411645412445068




Test MSE is 0.645636796951294


Train MSE is 0.23301681876182556

Test MSE is 0.6457853317260742


Train MSE is 0.2336110770702362

Test MSE is 0.6461013555526733


Train MSE is 0.23207984864711761

Test MSE is 0.6458600163459778


Train MSE is 0.23258212208747864

Test MSE is 0.6452816724777222


Train MSE is 0.2328941971063614

Test MSE is 0.6452112793922424


Train MSE is 0.23335935175418854

Test MSE is 0.6456452012062073


Train MSE is 0.23171499371528625

Test MSE is 0.6458771824836731


Train MSE is 0.23212316632270813

Test MSE is 0.6459618210792542


Train MSE is 0.23283828794956207

Test MSE is 0.645858883857727


Train MSE is 0.23197421431541443

Test MSE is 0.6456059217453003


Train MSE is 0.23257051408290863

Test MSE is 0.6455594301223755


Train MSE is 0.23254121840000153

Test MSE is 0.6456176042556763


Train MSE is 0.23266172409057617

Test MSE is 0.6458302736282349


Train MSE is 0.23140756785869598

Test MSE is 0.6464405655860901


Train MSE is 0.2325756996870041



Test MSE is 0.6494709849357605


Train MSE is 0.22235393524169922

Test MSE is 0.6491332054138184


Train MSE is 0.2230173945426941

Test MSE is 0.6488684415817261


Train MSE is 0.22315506637096405

Test MSE is 0.6486591696739197


Train MSE is 0.22280560433864594

Test MSE is 0.6489075422286987


Train MSE is 0.22232745587825775

Test MSE is 0.6495038270950317


Train MSE is 0.2228853851556778

Test MSE is 0.6500055193901062


Train MSE is 0.22257409989833832

Test MSE is 0.6499384641647339


Train MSE is 0.22145986557006836

Test MSE is 0.6492820382118225


Train MSE is 0.2217489331960678

Test MSE is 0.6488375663757324


Train MSE is 0.22149838507175446

Test MSE is 0.6489253640174866


Train MSE is 0.22115442156791687

Test MSE is 0.6494161486625671


Train MSE is 0.22079254686832428

Test MSE is 0.6501612663269043


Train MSE is 0.22085745632648468

Test MSE is 0.6502493619918823


Train MSE is 0.22101102769374847

Test MSE is 0.6501344442367554


Train MSE is 0.22114980220794678

Test MSE is 0.6515689492225647


Train MSE is 0.2144407033920288

Test MSE is 0.6520075798034668


Train MSE is 0.2125798910856247

Test MSE is 0.6530405879020691


Train MSE is 0.21330970525741577

Test MSE is 0.6534791588783264


Train MSE is 0.2148362249135971

Test MSE is 0.653262734413147


Train MSE is 0.21329930424690247

Test MSE is 0.6527236104011536


Train MSE is 0.21271181106567383

Test MSE is 0.6523177623748779


Train MSE is 0.21474221348762512

Test MSE is 0.6520692706108093


Train MSE is 0.21434807777404785

Test MSE is 0.6522648334503174


Train MSE is 0.2137177735567093

Test MSE is 0.6528527736663818


Train MSE is 0.21494145691394806

Test MSE is 0.6535687446594238


Train MSE is 0.21397201716899872

Test MSE is 0.6534736156463623


Train MSE is 0.2131592184305191

Test MSE is 0.6528215408325195


Train MSE is 0.21405597031116486

Test MSE is 0.6520158052444458


Train MSE is 0.21363310515880585

Test MSE is 0.6516426205635071


Train MSE is 0.21373778581619263

T

KeyboardInterrupt: 

In [150]:
torch.sum(lin.lin1.weight > 1e-5)

tensor(5093)

In [153]:
torch.sum(mylin.lin1.weight > 1e-5)

tensor(100)

In [128]:
lin.lin2.weight

Parameter containing:
tensor([[-0.0253, -0.0378, -0.0311,  ..., -0.0055, -0.0189,  0.0060],
        [-0.0210,  0.0146,  0.0259,  ...,  0.0096, -0.0153,  0.0153],
        [-0.0034,  0.0062, -0.0159,  ...,  0.0136, -0.0174,  0.0279],
        ...,
        [ 0.0219,  0.0045,  0.0042,  ..., -0.0146,  0.0144, -0.0402],
        [-0.0333,  0.0012,  0.0213,  ...,  0.0071,  0.0049, -0.0286],
        [-0.0145, -0.0189,  0.0289,  ...,  0.0229, -0.0258,  0.0081]],
       requires_grad=True)